In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.contrib.layers.python.layers.initializers import variance_scaling_initializer

import skimage.io as io
import numpy as np
from src.util import image as img_util
from matplotlib import pyplot as plt

from utilsTest import *

In [2]:
def Encoder_resnet(x, is_training=True, weight_decay=0.001, reuse=False):
    """
    Resnet v2-50
    Assumes input is [batch, height_in, width_in, channels]!!
    Input:
    - x: N x H x W x 3
    - weight_decay: float
    - reuse: bool->True if test

    Outputs:
    - cam: N x 3
    - Pose vector: N x 72
    - Shape vector: N x 10
    - variables: tf variables
    """
    from tensorflow.contrib.slim.python.slim.nets import resnet_v2
    # with tf.name_scope("Encoder_resnet", [x]):
    with tf.name_scope("Encoder_resnet"):
        with slim.arg_scope(resnet_v2.resnet_arg_scope(weight_decay=weight_decay)):
            net, end_points = resnet_v2.resnet_v2_50(
                x,
                num_classes=None,
                is_training=is_training,
                reuse=reuse,
                scope='resnet_v2_50')
            net = tf.squeeze(net, axis=[1, 2])
    variables = tf.contrib.framework.get_variables('resnet_v2_50')
    return net, variables

In [3]:
img_path = 'data/im1963.jpg'
json_path = None
input_img, proc_param, img = preprocess_image(img_path, json_path)
input_img = np.expand_dims(input_img, 0)
input_img = np.float32(input_img)

Resizing so the max image size is 224..


In [4]:
# img_enc_fn, threed_enc_fn = get_encoder_fn_separate('resnet_fc3_dropout')
# img_feat, E_var = img_enc_fn(input_img, is_training=False, reuse=False)

In [5]:
img_feat, E_var = Encoder_resnet(input_img, is_training=False, reuse=False)

In [6]:
img_feat

<tf.Tensor 'Encoder_resnet/Squeeze:0' shape=(1, 2048) dtype=float32>

In [ ]:
# E_var